In [1]:
! pip install setfit

  Using cached pillow-10.2.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (9.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 7.9 MB/s eta 0:00:00
Using cached pillow-10.2.0-cp311-cp311-macosx_11_0_arm64.whl (3.3 MB)


In [2]:
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitModel, SetFitTrainer
import pandas as pd
import numpy as np
import torch
from datasets import Dataset
import random
import os

In [3]:
# Check gpu
device = "mps" if torch.backends.mps.is_built() else "cuda" if torch.cuda.is_available() else "cpu"
print(device)

mps


In [4]:
# For reproducibility, you can define the following function to fix the random seeds.
def seed_everything(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)


seed_everything(42)

In [6]:
# Read the data
yelp_review = pd.read_csv('./data/train.csv', header=0)
# make the stars 0, 1, 2
yelp_review['stars'] = yelp_review['stars'] - 1

test_data = pd.read_csv('./data/new_test.csv', header=0)
test_label = list(test_data['stars'])
final_test_data = pd.read_csv('./data/test.csv', header=0)
final_test_data['ID'] = 0

yelp_review = yelp_review.rename(columns={'stars': 'label'})
test_data = test_data.rename(columns={'stars': 'label'})
final_test_data = final_test_data.rename(columns={'ID': 'label'})

dataset = Dataset.from_pandas(yelp_review)
test_dataset = Dataset.from_pandas(test_data)
fina_test_dataset = Dataset.from_pandas(final_test_data)

# Load SetFit model from Hub
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")

In [11]:
# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=dataset,
    eval_dataset=test_dataset,
    loss_class=CosineSimilarityLoss,
    batch_size=4,
    num_iterations=20, # Number of text pairs to generate for contrastive learning
    num_epochs=1 # Number of epochs to use for contrastive learning
)


/var/folders/7y/z1_q_y7n1rgcftmm9bnt0lt00000gn/T/ipykernel_2972/4267404778.py:2: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(


Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

In [12]:
# Train and evaluate!
trainer.train()
metrics = trainer.evaluate()

***** Running training *****
  Num unique pairs = 240000
  Batch size = 4
  Num epochs = 1
  Total optimization steps = 60000


Step,Training Loss


KeyboardInterrupt: 